In [1]:
from openai import OpenAI
import json

First, let's initialize the client:

In [2]:
client = OpenAI()

Here is the list of text messages we want to process:

In [3]:
text_messages = [
    "The service here is very good!",
    "The service here is good.",
    "The service here is ok.",
    "The service here is not very good.",
    "The service here is terrible!",
]

Now let's create the batch file.
We have some reuseable code for prompts from before.

In [4]:
system_prompt = "You are an expert on sentiment analysis. Your job is to evaluate the sentiment of the given text message."

user_instruction = """
    Given the following text message: '{text_message}', please evaluate its sentiment by giving a score in the range of -1 to 1, where -1 means negative and 1 means positive.
    Also explain why.
    """

Let's define the JSON schema for the response. For the batch API, you have to specify the schema manually.

In [5]:
sentiment_json_schema = {
    "type": "object",
    "title": "Sentiment",
    "required": ["score", "explanation"],
    "properties": {
        "score": {
            "type": "number",
            "title": "Score",
            "description": "Sentiment score in the range of -1 to 1, where -1 means negative and 1 means positive.",
        },
        "explanation": {
            "type": "string",
            "title": "Explanation",
            "description": "Explanation of the sentiment score.",
        },
    },
    "additionalProperties": False,
}

Let's define the tasks

In [6]:
tasks = []
for index, text_message in enumerate(text_messages):
    task = {
        # The API won't return the input text message, so we need a unique ID for each task
        # This way we can merge the results back with the input text message
        # Instead of generating the ID on the fly, it's recommended to assign a unique ID to each input message at the beginning
        "custom_id": f"text_message_{index}",
        "method": "POST",
        "url": "/v1/responses",
        "body": {
            # This is what you would have in your Chat Completions API call
            "model": "gpt-4o-mini",
            "temperature": 0.0,
            "instructions": system_prompt,
            "input": user_instruction.format(text_message=text_message),
            "text": {
                "format": {
                    "type": "json_schema",
                    "name": "sentiment",
                    "strict": True,
                    "schema": sentiment_json_schema,
                }
            },
        },
    }

    tasks.append(task)

Now we need to store the results in a file.
Each line is a JSON object.

In [7]:
task_file_name = "text_message_tasks.jsonl"
with open(task_file_name, "w") as f:
    for task in tasks:
        f.write(json.dumps(task) + "\n")

Now we can upload the task file.

In [8]:
batch_file = client.files.create(
    file=open(task_file_name, "rb"),
    purpose="batch"
)

In [9]:
print(batch_file)

FileObject(id='file-N8XHkV6uA3UmLxL5b4JgA1', bytes=4846, created_at=1747097239, filename='text_message_tasks.jsonl', object='file', purpose='batch', status='processed', expires_at=None, status_details=None)


We need to write the batch file id down.
It's used to track the status of the batch later.

After this step, you can also view the file at https://platform.openai.com/storage.
Be sure to switch the organization/project associated with your API key.


Now we can create the batch job:

In [10]:
batch_job = client.batches.create(
    input_file_id=batch_file.id,
    endpoint="/v1/responses",
    completion_window="24h"
)

This step can also be done at https://platform.openai.com/batch using the UI.

In [21]:
print(batch_job)

Batch(id='batch_682296a634348190850d5ffcd04ba83e', completion_window='24h', created_at=1747097254, endpoint='/v1/responses', input_file_id='file-N8XHkV6uA3UmLxL5b4JgA1', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1747183654, failed_at=None, finalizing_at=None, in_progress_at=None, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))


Let's check the status of the batch job

In [22]:
batch_object = client.batches.retrieve(batch_job.id)

In [23]:
batch_object.status

'completed'

After the batch job is done, we can download the results.

In [24]:
result_file_id = batch_object.output_file_id
result = client.files.content(result_file_id).content

In [25]:
result_file_name = "openai_text_message_batch_output.jsonl"

with open(result_file_name, 'wb') as file:
    file.write(result)

Now we can read the results.

In [26]:
results = []
with open("openai_text_message_batch_output.jsonl") as f:
    for line in f:
        results.append(json.loads(line))

In [30]:
print(
    results[0]['response']['body']['output'][0]['content'][0]['text']
)

{"score":0.9,"explanation":"The text message expresses a positive sentiment about the service, using the phrase 'very good' which indicates a strong approval. The overall tone is enthusiastic and favorable, leading to a high positive score."}
